In [1]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [9]:
#Imports
import joblib
import sklearn
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

## Carregando o Conjunto de Dados

In [5]:
df=pd.read_csv('dataset.csv')
df.head()

,data,indice_vegetacao,capacidade_solo,concentracao_co2,nivel_nutrientes,indice_fertilizantes,profundidade_raiz,radiacao_solar,precipitacao,estagio_crescimento,historico_rendimento,umidade
0,2012-12-01,323,455,3102.61,423.45,844.0,468.0,578.0,28.67,207.70504,117.7,79.261905
1,2013-01-01,345,546,3100.45,415.85,799.0,485.0,557.0,24.49,228.94287,4.5,82.193548
2,2013-02-01,362,595,3199.41,410.77,718.0,466.0,552.0,22.06,238.41747,25.1,74.839286
3,2013-03-01,376,636,3281.67,414.82,614.0,442.0,574.0,21.64,218.47599,53.6,77.935484
4,2013-04-01,383,738,3261.65,451.04,619.0,429.0,595.0,22.30,226.15010,166.0,80.450000


In [6]:
df.shape

(124, 12)

In [7]:
df.tail()

,data,indice_vegetacao,capacidade_solo,concentracao_co2,nivel_nutrientes,indice_fertilizantes,profundidade_raiz,radiacao_solar,precipitacao,estagio_crescimento,historico_rendimento,umidade
119,2022-11-01,362,363,2626.91,1252.78,738.07,427.49,1430.48,60.18,186.68326,38.2,77.950000
120,2022-12-01,310,322,2736.64,1287.68,749.57,385.09,1472.27,62.25,210.72987,33.7,76.177419
121,2023-01-01,277,307,2842.81,1289.12,761.60,373.03,1525.43,63.04,244.41912,4.6,74.774194
122,2023-02-01,323,330,2936.19,1303.59,759.59,390.69,1572.25,71.52,223.31732,6.9,66.910714
123,2023-03-01,360,339,2847.84,1234.88,771.62,396.87,1302.61,74.80,228.56676,41.5,69.000000


## Análise Exploratória

In [8]:
# Verificar os tipos de dados das colunas para comparar com o dicionario
df.dtypes

data                     object
indice_vegetacao          int64
capacidade_solo           int64
concentracao_co2        float64
nivel_nutrientes        float64
indice_fertilizantes    float64
profundidade_raiz       float64
radiacao_solar          float64
precipitacao            float64
estagio_crescimento     float64
historico_rendimento    float64
umidade                 float64
dtype: object

In [11]:
# Exibir as colunas do dataset para ver se tem algum problema de espaço no nome ou algum outro tipo de problema comum
df.columns

Index(['data', 'indice_vegetacao', 'capacidade_solo', 'concentracao_co2',
       'nivel_nutrientes', 'indice_fertilizantes', 'profundidade_raiz',
       'radiacao_solar', 'precipitacao', 'estagio_crescimento',
       'historico_rendimento', 'umidade'],
      dtype='object')

In [13]:
# Colunas não numéricas precisam ser tratadas
non_numeric_columns = df.select_dtypes(include = ['object']).columns
print(f'Colunas não numéricas: {non_numeric_columns}')

Colunas não numéricas: Index(['data'], dtype='object')


In [16]:
#Verificar se tem valores nulos
df.isnull().sum()

data                    0
indice_vegetacao        0
capacidade_solo         0
concentracao_co2        0
nivel_nutrientes        0
indice_fertilizantes    0
profundidade_raiz       0
radiacao_solar          0
precipitacao            0
estagio_crescimento     0
historico_rendimento    0
umidade                 0
dtype: int64

## Limpeza e Transformação

In [18]:
# A data nesse dataset está servindo como um ID. Como eu não estou fazendo uma análise de série temporal, irei remover essa coluna
df = df.drop(columns = non_numeric_columns)

## Padronização dos Dados

In [19]:
# Define a variável alvo
X = df.drop(columns='umidade')
y = df['umidade']

In [20]:
# Separa os dados em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [22]:
# Define o padronizador
scaler = StandardScaler()

In [32]:
# Padroniza os dados
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

In [25]:
# Salva o padronizador em disco, pois ele será utilizado nos novos dados após o deploy
joblib.dump(scaler,'scaler.joblib')

['scaler.joblib']

In [27]:
# Definir a arquitetura do modelo. 
# Utilizei uma arquitetura típica em problemas de regressão onde se deseja prever um único valor contínuo a partir de multiplas características de entrada
modelo = Sequential([Dense(64, activation = 'relu', input_shape = (X_treino.shape[1],)),
                         Dropout(0.3),
                         Dense(32, activation = 'relu'),
                         Dropout(0.3),
                         Dense(16, activation = 'relu'),
                         Dense(1)])

In [28]:
# Compilar o modelo
modelo_dsa.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

In [29]:
# Callbacks. Esses callbacks servem para definir quando o modelo deve parar de ser treinado e para salvar toda versão que for melhor que a anterior
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)
model_checkpoint = ModelCheckpoint('modelo_dsa.keras', save_best_only = True)

## Treinamento do Modelo

In [30]:
modelo.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 64)                  │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,329 (13.00 KB)

 Trainable params: 3,329 (13.00 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Treinar o modelo
history = modelo_dsa.fit(X_treino_scaled, 
                         y_treino,
                         validation_split = 0.2,
                         epochs = 100,
                         batch_size = 32,
                         callbacks = [early_stopping, model_checkpoint])

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - loss: 5227.2500 - mae: 72.1609 - val_loss: 5194.6377 - val_mae: 71.9685
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 5182.2324 - mae: 71.8521 - val_loss: 5173.6763 - val_mae: 71.8229
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 5168.2393 - mae: 71.7569 - val_loss: 5152.1108 - val_mae: 71.6727
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 5169.2808 - mae: 71.7564 - val_loss: 5128.9414 - val_mae: 71.5110
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 5132.2896 - mae: 71.5120 - val_loss: 5104.1958 - val_mae: 71.3378
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 5133.6685 - mae: 71.5235 - val_loss: 5076.8149 - val_mae: 71.1456
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 5056.2051 - mae: 70.9758 - val_loss: 5046.0000 - val_mae: 70.9289
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 5049.5918 - mae: 70.9121 - val_loss: 5011.4248 - val_mae: 70.6849
Epoch 9

## Avaliação do Modelo

In [36]:
# Avaliar o modelo no conjunto de teste
teste_loss, teste_mae = modelo_dsa.evaluate(X_teste_scaled, y_teste)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 163.9450 - mae: 10.5536


In [37]:
print(f'Teste Loss: {teste_loss}')
print(f'Teste MAE: {teste_mae}')

Teste Loss: 163.94497680664062
Teste MAE: 10.553627014160156
